# Import articles from macrostructure.json

In [1]:
import json

with open("resources/macrostructure.json", "r") as macrostructure_file:
    articles_list = json.load(macrostructure_file)

# Create Ontology

In [2]:
from owlready2 import owlready2, get_ontology, sync_reasoner_pellet, Thing, Not, And, default_world, AllDifferent, AllDisjoint, Imp

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
owlready2.JAVA_EXE='/usr/bin/java'

In [4]:
# Fetch root ontology
onto = get_ontology("http://authors_relations.org/onto.owl")

In [5]:
# Create ontology classes
with onto :
    class Author(Thing):
        pass

    class Article(Thing):
        # uri of an article is its uri in arxiv
        is_a = [Not(onto.Author)]

    class is_author_of(Author >> Article): pass

    class is_written_by(Article >> Author): pass
    
    class cites(Author >> Author): pass

    class worked_with(Author >> Author): pass

    class is_cited_by(Author >> Author):
        inverse_property = cites

    class is_cited_in(Author >> Article): pass

    class CitedAuthor(Thing):
        # as a Person who teaches some Course
        equivalent_to = [And([Author, is_cited_by.some(Author), is_cited_in.some(Article)])]

In [6]:
# Data preprocessing



In [ ]:
from cleaner_utils.cleaner_custom import escape_iri, escape_value

# Populating individuals in ontology
with onto:
    for article in articles_list:
        uri = article['uri']
        urn = uri.replace("http://arxiv.org/pdf/cs/","")
        article_object = onto.Article(
            urn=escape_iri(urn),
            )
        for article_author in article['authors']:
            try:
                # Look for article_author (if already created in ontology)
                author_object = onto.search(iri="*{}".format(escape_iri(article_author)))[0]
            except IndexError:
                # Create it otherwise
                author_object = onto.Author(
                    name=escape_iri(article_author)
                )
            article_object.is_written_by.append(author_object)
        for reference in article['references']:
            for cited_author in reference['authors']:
                try:
                    # Look for cited_author (if already created in ontology)
                    cited_author_object = onto.search(iri="*{}".format(escape_iri(cited_author)))[0]
                except IndexError:
                    # Create it otherwise
                    cited_author_object = onto.Author(
                        name=escape_iri(cited_author)
                    )
                cited_author_object = onto.Author(
                    name=escape_iri(cited_author)
                )
                if article_object not in cited_author_object.is_cited_in:
                    cited_author_object.is_cited_in.append(article_object)
            
    print(onto.Article.instances())

In [8]:
# Defining reasoner rules


In [ ]:
# Applying rules by reasoning

In [ ]:
# Save the ontology
onto.save(file = "authors_relations.owl")

In [ ]:
# Case permet de réinitialiser les instances enregistrées
for individual in Author.instances(): 
    destroy_entity(individual)

for individual in ReferencedAuthor.instances(): 
    destroy_entity(individual)

for individual in Article.instances(): 
    destroy_entity(individual)